In [2]:
install.packages("randomForest")

Installing package into 'C:/Users/lopez/AppData/Local/R/win-library/4.4'
(as 'lib' is unspecified)



package 'randomForest' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\lopez\AppData\Local\Temp\RtmpC81pUc\downloaded_packages


In [1]:
library(randomForest)
library(dplyr)
library(readr)
library(e1071)
library(caret)

Warning message:
"package 'randomForest' was built under R version 4.4.2"
randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Adjuntando el paquete: 'dplyr'


The following object is masked from 'package:randomForest':

    combine


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Cargando paquete requerido: ggplot2


Adjuntando el paquete: 'ggplot2'


The following object is masked from 'package:randomForest':

    margin


Cargando paquete requerido: lattice



In [2]:
df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-train.csv', show_col_types = FALSE)
test_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mza-dataset-test.csv', show_col_types = FALSE)
val_df <- read_csv('https://raw.githubusercontent.com/gabichulas/ia-uncuyo-2024/refs/heads/main/tp7-ml/data/arbolado-mendoza-dataset-validation.csv', show_col_types = FALSE)

In [3]:
set_1 <- df[df$inclinacion_peligrosa == 1, ]
set_0 <- df %>% filter(inclinacion_peligrosa == 0) %>% sample_n(3500)
data <- rbind(set_1, set_0)
table(data$inclinacion_peligrosa)


   0    1 
3500 2895 

In [ ]:
x <- data[, !(colnames(data) %in% c("inclinacion_peligrosa", "id", "area_seccion","seccion", "ultima_modificacion"))]
y <- factor(data$inclinacion_peligrosa)

In [ ]:
model <- randomForest(x = x, y = y, ntree = 10000, mtry = 2)
print(model)

predicciones_test <- predict(model, newdata = val_df)

etiquetas_reales_test <- val_df$inclinacion_peligrosa

confusion_matrix_test <- table(Predicted = predicciones_test, Actual = etiquetas_reales_test)

precision <- sum(diag(confusion_matrix_test)) / sum(confusion_matrix_test)

print(confusion_matrix_test)
cat("Precisión: ", precision, "\n")


Call:
 randomForest(x = x, y = y, ntree = 10000, mtry = 2) 
               Type of random forest: classification
                     Number of trees: 10000
No. of variables tried at each split: 2

        OOB estimate of  error rate: 30.24%
Confusion matrix:
     0    1 class.error
0 2450 1050   0.3000000
1  884 2011   0.3053541
         Actual
Predicted    0    1
        0 4002  211
        1 1696  473
Precisión:  0.7011908 
Recall:  0.9499169 0.2180728 
F1-Score:  0.8068196 0.3326808 


In [74]:
predicciones <- predict(model, newdata = test_df)

submission <- data.frame(ID = test_df$id, inclinacion_peligrosa = predicciones)

write.csv(submission, file = "submission.csv", row.names = FALSE)

print("Predicciones guardadas en submission.csv")

[1] "Predicciones guardadas en submission.csv"
